# Abates - Análise

Pergunta: quanto de gado vou precisar engordar para abater no último trimestre de 2021?

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

for dirname, _, filenames in os.walk('../input/abates'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Descartando o uso das datasets do preço das carcaças
peso_carcacas_files = [
    '../input/abates/2020/peso_total_carcacas.csv',
    '../input/abates/2019/peso_total_carcacas.csv',
    '../input/abates/2018/peso_total_carcacas.csv',
    '../input/abates/2017/peso_total_carcacas.csv',
]

abates_files = [
    '../input/abates/2020/numero_abates.csv',
    '../input/abates/2019/numero_abates.csv',
    '../input/abates/2018/numero_abates.csv',
    '../input/abates/2017/numero_abates.csv',
]

abates_datasets = []

for file in abates_files:
    aux_dataset = pd.read_csv(file)
    abates_datasets.append(aux_dataset)

In [ ]:
abates_datasets[0].head()

In [ ]:
# Função que normaliza as colunas das tabelas
def prepare_name_columns(dataset):
    # Normalizando os nomes das colunas
    for column in dataset.columns:
        dataset.rename(columns={
            column: column
                .replace(' ', '_')
                .replace('ç', 'c')
                .replace('ã', 'a')
                .replace('í', 'i')
                .lower(),
        }, inplace=True)
    
    # Removendo espaços em brancos da coluna 'tempo'
    dataset['tempo'] = dataset['tempo'].str.strip()
    dataset['tempo'] = dataset['tempo'].str.lower()

# Função que remove linhas desnecessárias
def remove_rows(dataset):
    # Removendo linhas que tenham "Total"
    new_dataset = dataset[~dataset['tempo'].str.contains('total')]
    new_dataset = new_dataset.replace(',','.', regex=True)
    new_dataset.reset_index(inplace=True)
    return new_dataset

for index, dataset in enumerate(abates_datasets):
    prepare_name_columns(dataset)
    new_dataset = remove_rows(dataset)
    abates_datasets[index] = new_dataset

abates_bovinos_2020 = abates_datasets[0][['tempo', 'bovinos_2019', 'bovinos_2020', 'bovinos_variacao']]
abates_bovinos_2019 = abates_datasets[1][['tempo', 'bovinos_2018', 'bovinos_2019', 'bovinos_variacao']]
abates_bovinos_2018 = abates_datasets[2][['tempo', 'bovinos_2017', 'bovinos_2018', 'bovinos_variacao']]
abates_bovinos_2017 = abates_datasets[3][['tempo', 'bovinos_2016', 'bovinos_2017', 'bovinos_variacao']]

# Fazendo merge de dois datasets em uma coluna
def merge_datasets(left_dataset, right_dataset, column):
    return pd.merge(
        left=left_dataset,
        right=right_dataset,
        left_on=column,
        right_on=column,
    )

abates_dataset = merge_datasets(abates_bovinos_2017, abates_bovinos_2018, 'tempo')
abates_dataset = merge_datasets(abates_dataset, abates_bovinos_2019, 'tempo')
abates_dataset = merge_datasets(abates_dataset, abates_bovinos_2020, 'tempo')
abates_dataset.head()

In [ ]:
# Convertendo os valores nas colunas em FLOAT
columns = abates_dataset.columns[1:]
for column in columns.values:
    abates_dataset[column] = abates_dataset[column].astype(float)

# Função responsável por fazer a média de duas colunas correspondentes ao memso ano
def media_by_year(dataset, column_default, column_x, column_y):
    dataset[column_default] = (
        (dataset[column_x] + dataset[column_y]) / 2
    )
    dataset.drop(column_x, inplace=True, axis=1)
    dataset.drop(column_y, inplace=True, axis=1)

media_by_year(abates_dataset, 'bovinos_2017', 'bovinos_2017_x', 'bovinos_2017_y')
media_by_year(abates_dataset, 'bovinos_2018', 'bovinos_2018_x', 'bovinos_2018_y')
media_by_year(abates_dataset, 'bovinos_2019', 'bovinos_2019_x', 'bovinos_2019_y')

# Descartando as colunas variação. Não serão necessárias
abates_dataset.drop('bovinos_variacao_x', inplace=True, axis=1)
abates_dataset.drop('bovinos_variacao_y', inplace=True, axis=1)

for column in abates_dataset.columns:
    abates_dataset.rename(columns={column: column.replace('bovinos_','')}, inplace=True)

# Invertendo o dataset para que colunas virem linhas e linhas virem colunas
abates_dataset = abates_dataset.T
new_header = abates_dataset.iloc[0]
abates_dataset = abates_dataset.iloc[1:]
abates_dataset.columns = new_header

# Convertendo os valores nas colunas em FLOAT
columns = abates_dataset.columns
for column in columns.values:
    abates_dataset[column] = abates_dataset[column].astype(float)

# Ajustando o INDEX do dataset
abates_dataset.reset_index(inplace=True)
abates_dataset.rename(columns={
    'index': 'tempo',
}, inplace=True)

# Convertendo os valores da coluna 'tempo' em DateTime
abates_dataset['tempo'] =  pd.to_datetime(abates_dataset['tempo'], format='%Y')
abates_dataset.head()

In [ ]:
sns.heatmap(abates_dataset.corr(),cbar=False,cmap='BuGn',annot=True)

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(abates_dataset.corr(), annot=True, linewidths=0.01, cmap='coolwarm', cbar=False)
plt.show()

In [ ]:
p=abates_dataset.hist(figsize = (20, 20))

In [ ]:
columns = ['tempo','outubro', 'novembro', 'dezembro']
quarto_trimestre = abates_dataset[columns]
quarto_trimestre.head()

In [ ]:
sns.pairplot(data=quarto_trimestre, kind="reg")

In [ ]:
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
scaler = preprocessing.MinMaxScaler()
regressao = LinearRegression()

ano = 2022

X = np.array(abates_dataset['tempo']).reshape(-1, 1)
X = pd.DataFrame(scaler.fit_transform(X), columns=['tempo'])

def make_prediction(dataset, regressao, X, column_y):
    Y = le.fit_transform(dataset[column_y])
    regressao.fit(X, Y)
    plt.scatter(X, Y, s=3)
    plt.plot(X, (X * regressao.coef_ + regressao.intercept_), color='r')
    print(column_y, regressao.predict(np.array([ano]).reshape(-1, 1)))
    plt.show()

for column in quarto_trimestre.columns[1:]:
    make_prediction(quarto_trimestre, regressao, X, column)